# 🧠 Dendritic YOLOv8: PerforatedAI Hackathon Submission

This notebook demonstrates applying **PerforatedAI's dendritic optimization** to YOLOv8n for improved efficiency on edge devices.

## Overview
1. **Setup** - Install dependencies and configure environment
2. **Baseline Training** - Train standard YOLOv8n on COCO128
3. **Dendritic Training** - Apply PerforatedAI optimization and retrain
4. **Comparison** - Analyze metrics and visualize improvements

---
## Section A: Setup
Install all required dependencies and configure the environment.

### 🚨 IMPORTANT GPU Setup Instructions

**For Google Colab users:**
1. Before running any cells, go to: **Runtime → Change runtime type**
2. Select **T4 GPU** or **A100 GPU** (if available)
3. Click **Save**
4. **Restart the runtime** if needed
5. Then run the cells below in order

**If you see PyTorch CPU version instead of CUDA:**
- Restart runtime completely: **Runtime → Restart runtime**
- Re-run cells from the beginning
- Do NOT install torch/torchvision manually - use Colab's pre-installed CUDA version

In [ ]:
# Install dependencies (Colab-specific setup for GPU)
# In Colab: Runtime → Change runtime type → Select T4 GPU first!

# Don't install torch/torchvision - use Colab's pre-installed CUDA version
!pip install ultralytics wandb matplotlib pandas seaborn --quiet
!pip install perforatedai==3.0.7 --quiet

# PyTorch checkpoint loading patch - IDEMPOTENT (prevents recursion)
import torch

# Safe idempotent patch - only apply once, even if cell is re-run
if not hasattr(torch, '_original_load_backup'):
    torch._original_load_backup = torch.load
    def torch_load_patched(*args, **kwargs):
        kwargs["weights_only"] = False
        return torch._original_load_backup(*args, **kwargs)
    torch.load = torch_load_patched
    print("✅ torch.load patched for weights_only=False")
else:
    print("ℹ️ torch.load patch already applied (skipping)")

print(f"✅ Dependencies installed!")
print(f"✅ PyTorch {torch.__version__} (CUDA: {torch.cuda.is_available()})")

# Verify we have GPU version
if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ CPU version detected. In Colab: Runtime → Change runtime type → GPU")

In [ ]:
# GPU verification and device setup
import torch
import subprocess

def check_nvidia_gpu():
    """Check for NVIDIA GPU availability across different environments."""
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, shell=True)
        if result.returncode == 0:
            print("✅ NVIDIA GPU detected:")
            # Show relevant GPU info lines
            lines = result.stdout.split('\n')
            for line in lines[8:12]:
                if line.strip():
                    print(f"   {line}")
            return True
        else:
            print("❌ nvidia-smi command failed")
            return False
    except FileNotFoundError:
        print("❌ nvidia-smi not found - NVIDIA drivers may not be installed")
        return False

# Setup device with proper error handling
if torch.cuda.is_available():
    device = 'cuda'
    gpu_detected = check_nvidia_gpu()
    print(f"\n✅ PyTorch CUDA available! Using device: {device}")
    try:
        print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
        print(f"   CUDA Version: {torch.version.cuda}")
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    except Exception as e:
        print(f"   GPU info error: {e}")
else:
    device = 'cpu'
    print(f"\n⚠️ CUDA not available. Using device: {device}")
    print("📋 To fix this in Colab:")
    print("   1. Runtime → Change runtime type")
    print("   2. Select 'T4 GPU' or 'A100 GPU'")
    print("   3. Click Save, then restart runtime")
    print("   4. Re-run cells from the beginning")

print(f"\n🎯 Device configured: {device}")

In [32]:
# Login to Weights & Biases 
import wandb
import os
from getpass import getpass

# Option 1: Use environment variable if set
if "WANDB_API_KEY" in os.environ:
    api_key = os.environ["WANDB_API_KEY"]
    print("✅ Using WANDB_API_KEY from environment")
else:
    # Option 2: Prompt for API key (more secure for sharing notebooks)
    api_key = getpass("Enter your W&B API key (get it from https://wandb.ai/authorize): ")
    os.environ["WANDB_API_KEY"] = api_key

try:
    wandb.login(key=api_key)
    print("✅ W&B authenticated successfully!")
except Exception as e:
    print(f"❌ W&B authentication failed: {e}")
    print("Note: You can skip W&B logging by setting WANDB_MODE=disabled")
    print("      Or run: wandb offline")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


✅ Using WANDB_API_KEY from environment
✅ W&B authenticated successfully!


In [33]:
# Import all required libraries
import os
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO

# PerforatedAI imports with error handling
try:
    from perforatedai import globals_perforatedai as GPA
    from perforatedai import utils_perforatedai as UPA
    print("✅ PerforatedAI imported successfully!")
    PERFORATED_AI_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ PerforatedAI not available: {e}")
    print("Note: This notebook will run in baseline mode only without dendritic optimization")
    PERFORATED_AI_AVAILABLE = False
    # Create dummy objects to prevent errors
    class DummyGPA:
        class pc:
            @staticmethod
            def set_testing_dendrite_capacity(val): pass
            @staticmethod
            def set_verbose(val): pass
            @staticmethod
            def set_dendrite_update_mode(val): pass
        class pai_tracker:
            @staticmethod
            def set_optimizer(opt): pass
            @staticmethod
            def set_scheduler(sched): pass
            @staticmethod
            def setup_optimizer(model, opt_args, sched_args): 
                import torch.optim as optim
                return optim.Adam(model.parameters(), **opt_args), None
    
    class DummyUPA:
        @staticmethod
        def initialize_pai(model, **kwargs):
            return model
    
    GPA = DummyGPA()
    UPA = DummyUPA()

print("✅ All imports successful!")

✅ PerforatedAI imported successfully!
✅ All imports successful!


---
## Section B: Baseline Training
Train standard YOLOv8n on COCO128 dataset to establish baseline metrics.

In [34]:
# Helper function to count parameters
def count_parameters(model):
    """Count total and trainable parameters in a model."""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

# Helper function to measure inference speed
def measure_inference_speed(model, img_size=640, num_runs=100):
    """Measure average inference time in milliseconds."""
    model.eval()
    dummy_input = torch.randn(1, 3, img_size, img_size).to(device)
    
    # Warmup
    for _ in range(10):
        with torch.no_grad():
            _ = model(dummy_input)
    
    # Measure
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(num_runs):
        with torch.no_grad():
            _ = model(dummy_input)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    end = time.perf_counter()
    avg_time_ms = (end - start) / num_runs * 1000
    return avg_time_ms

print("✅ Helper functions defined!")

✅ Helper functions defined!


In [35]:
# Initialize W&B for baseline run
wandb.init(
    project="Dendritic-YOLOv8-Hackathon",
    name="baseline-yolov8n",
    tags=["baseline", "yolov8n", "coco128"],
    config={
        "model": "yolov8n",
        "dataset": "coco128",
        "epochs": 5,
        "optimization": "none"
    }
)

print("✅ W&B initialized for baseline run")

baseline_params_M,▁
baseline_params_M,3.1572


✅ W&B initialized for baseline run


In [ ]:
# Load baseline YOLOv8n model with error handling
print("🚀 Loading YOLOv8n baseline model...")

try:
    baseline_model = YOLO("yolov8n.pt")
    print("✅ Model loaded successfully!")
    
    # Move model to device
    baseline_model.model = baseline_model.model.to(device)
    
    # Get baseline parameter count - access the actual PyTorch model
    model_params = baseline_model.model
    baseline_total_params, baseline_trainable_params = count_parameters(model_params)
    
    print(f"📊 Baseline Parameters: {baseline_total_params / 1e6:.2f}M total, {baseline_trainable_params / 1e6:.2f}M trainable")
    print(f"📱 Model device: {next(model_params.parameters()).device}")
    
    # Log to W&B if available
    try:
        wandb.log({"baseline_params_M": baseline_total_params / 1e6})
        print("✅ Logged to W&B")
    except:
        print("ℹ️ W&B logging skipped")
        
except Exception as e:
    print(f"❌ Model loading error: {e}")
    print("\n🔧 Troubleshooting:")
    print("   - Try restarting the runtime (Runtime → Restart runtime)")
    print("   - Ensure GPU is selected if available")
    print("   - Re-run cells from the beginning")
    
    # Use fallback values for demonstration
    baseline_total_params = 3157200  # YOLOv8n typical param count
    baseline_trainable_params = 3157200
    print(f"\n📊 Using fallback baseline params: {baseline_total_params / 1e6:.2f}M")
    baseline_model = None  # Mark as unavailable

In [ ]:
# Train baseline model with robust error handling
print("🚀 Starting baseline training...")

if baseline_model is not None:
    try:
        baseline_results = baseline_model.train(
            data="coco128.yaml",
            epochs=5,
            imgsz=640,
            batch=16,
            device=device,  # Use our configured device
            project="runs/baseline", 
            name="yolov8n_coco128",
            exist_ok=True,
            verbose=True,
            save_period=5  # Save every 5 epochs
        )
        print("✅ Baseline training completed successfully!")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        print("🔧 This may be due to:")
        print("   - Insufficient GPU memory")
        print("   - Model loading issues")
        print("   - Dataset download problems")
        print("\n💡 Try reducing batch size to 8 or 4 if memory issues persist")
        baseline_results = None
        
else:
    print("⚠️ Skipping baseline training - model not loaded properly")
    print("💡 Fix model loading issues first, then retry training")
    baseline_results = None

In [ ]:
# Validate baseline model and extract metrics
print("📊 Validating baseline model...")

if baseline_model is not None and baseline_results is not None:
    try:
        baseline_val = baseline_model.val(
            data="coco128.yaml",
            device=device  # Use our configured device
        )

        # Extract metrics with comprehensive error handling
        baseline_metrics = {}
        try:
            baseline_metrics = {
                "mAP50": float(baseline_val.box.map50) if baseline_val.box.map50 is not None else 0.0,
                "mAP50-95": float(baseline_val.box.map) if baseline_val.box.map is not None else 0.0,
                "precision": float(baseline_val.box.mp) if baseline_val.box.mp is not None else 0.0,
                "recall": float(baseline_val.box.mr) if baseline_val.box.mr is not None else 0.0,
                "params_M": baseline_total_params / 1e6,
            }
        except Exception as e:
            print(f"⚠️ Error extracting validation metrics: {e}")
            baseline_metrics = {
                "mAP50": 0.0,
                "mAP50-95": 0.0,
                "precision": 0.0,
                "recall": 0.0,
                "params_M": baseline_total_params / 1e6,
            }

        # Measure inference speed
        try:
            baseline_metrics["inference_ms"] = measure_inference_speed(baseline_model.model)
        except Exception as e:
            print(f"⚠️ Error measuring inference speed: {e}")
            baseline_metrics["inference_ms"] = 0.0

        print(f"\n📊 Baseline Metrics:")
        for key, value in baseline_metrics.items():
            print(f"   {key}: {value:.4f}")

        # Log to W&B if available
        try:
            wandb.log({f"baseline_{k}": v for k, v in baseline_metrics.items()})
            wandb.finish()
            print("✅ Logged to W&B")
        except:
            print("ℹ️ W&B logging skipped")
            
    except Exception as e:
        print(f"❌ Validation failed: {e}")
        # Create minimal baseline metrics for comparison
        baseline_metrics = {
            "params_M": baseline_total_params / 1e6,
            "mAP50": 0.0,
            "mAP50-95": 0.0,
            "inference_ms": 0.0
        }
        
else:
    print("⚠️ Skipping baseline validation - training incomplete")
    # Create fallback metrics
    baseline_metrics = {
        "params_M": baseline_total_params / 1e6,
        "mAP50": 0.0,
        "mAP50-95": 0.0,
        "inference_ms": 0.0
    }

print("\n✅ Baseline evaluation complete!")

---
## Section C: Dendritic Training
Apply PerforatedAI's dendritic optimization to YOLOv8n and retrain.

In [ ]:
# Initialize W&B for dendritic run
try:
    wandb.init(
        project="Dendritic-YOLOv8-Hackathon",
        name="dendritic-yolov8n",
        tags=["dendritic", "perforatedai", "yolov8n", "coco128"],
        config={
            "model": "yolov8n",
            "dataset": "coco128",
            "epochs": 5,
            "optimization": "perforatedai_dendritic" if PERFORATED_AI_AVAILABLE else "baseline"
        }
    )
    print("✅ W&B initialized for dendritic run")
except:
    print("⚠️ W&B initialization skipped")

In [ ]:
# Load fresh YOLOv8n model for dendritic optimization
dendritic_yolo = YOLO("yolov8n.pt")
dendritic_model = dendritic_yolo.model

print("Model structure before optimization:")
print(dendritic_model)

In [ ]:
# Configure PerforatedAI settings
GPA.pc.set_testing_dendrite_capacity(False)
GPA.pc.set_verbose(True)
GPA.pc.set_dendrite_update_mode(True)

print("✅ PerforatedAI configuration set")

In [ ]:
# Apply dendritic optimization (if PerforatedAI is available)
print("🧠 Applying dendritic optimization...")

if PERFORATED_AI_AVAILABLE:
    # Save the input stem before optimization
    input_stem = dendritic_model.model[0]
    
    # Apply PerforatedAI initialization to the model
    try:
        dendritic_model = UPA.initialize_pai(
            dendritic_model,
            doing_pai=True,
            save_name="DendriticYOLOv8",
            maximizing_score=True
        )
        
        # Restore input stem to avoid weight loading issues
        dendritic_model.model[0] = input_stem
        print("✅ Dendritic optimization applied!")
        
    except Exception as e:
        print(f"⚠️ PerforatedAI optimization failed: {e}")
        print("Continuing with standard model...")
        
else:
    print("⚠️ PerforatedAI not available - using standard model")

dendritic_model = dendritic_model.to(device)

# Count parameters after optimization
dendritic_total_params, dendritic_trainable_params = count_parameters(dendritic_model)
print(f"📊 Dendritic Parameters: {dendritic_total_params / 1e6:.2f}M total, {dendritic_trainable_params / 1e6:.2f}M trainable")

try:
    wandb.log({"dendritic_params_M": dendritic_total_params / 1e6})
except:
    print("⚠️ W&B logging skipped")

In [ ]:
# Setup optimizer through PerforatedAI tracker
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

GPA.pai_tracker.set_optimizer(optim.Adam)
GPA.pai_tracker.set_scheduler(ReduceLROnPlateau)

optimArgs = {'params': dendritic_model.parameters(), 'lr': 1e-3}
schedArgs = {'mode': 'max', 'patience': 3, 'factor': 0.5}

optimizer, scheduler = GPA.pai_tracker.setup_optimizer(dendritic_model, optimArgs, schedArgs)

print("✅ Optimizer and scheduler configured through PerforatedAI")

In [ ]:
# Custom training loop with PerforatedAI integration
from ultralytics.data import build_dataloader, build_yolo_dataset
from ultralytics.utils import LOGGER
import torch.nn.functional as F

# Training configuration
EPOCHS = 5
BATCH_SIZE = 16
IMG_SIZE = 640

print(f"🚀 Starting dendritic training for {EPOCHS} epochs...")
print("Note: Using custom training loop with PerforatedAI integration")

In [ ]:
# Train the dendritic model with comprehensive error handling
print("🚀 Starting dendritic training with PerforatedAI optimization...")

if baseline_model is not None and dendritic_model is not None:
    # Re-assign the modified model back to the YOLO wrapper
    dendritic_yolo.model = dendritic_model

    try:
        dendritic_results = dendritic_yolo.train(
            data="coco128.yaml",
            epochs=5,
            imgsz=640,
            batch=16,
            device=device,  # Use our configured device
            project="runs/dendritic",
            name="yolov8n_dendritic_coco128",
            exist_ok=True,
            verbose=True,
            optimizer="Adam",
            lr0=0.001,
            save_period=5,  # Save checkpoints
            patience=50     # Early stopping patience
        )
        print("✅ Dendritic training completed successfully!")
        
    except Exception as e:
        print(f"❌ Dendritic training failed: {e}")
        print("🔧 This may be due to:")
        print("   - PerforatedAI model modifications")
        print("   - GPU memory constraints")
        print("   - Compatibility issues")
        print("\n💡 Continuing with validation of current model state...")
        dendritic_results = None
        
else:
    print("⚠️ Skipping dendritic training - prerequisites not met")
    if baseline_model is None:
        print("   - Baseline model not loaded")
    if 'dendritic_model' not in locals() or dendritic_model is None:
        print("   - Dendritic model not initialized")
    dendritic_results = None

In [ ]:
# === ADD VALIDATION SCORE (CRITICAL for PAI.png generation) ===
# This call is REQUIRED to:
# 1. Enable dendrite restructuring
# 2. Generate the PAI.png results graph

print("🧠 Adding validation score to PerforatedAI tracker...")

if PERFORATED_AI_AVAILABLE:
    # Get validation score
    val_results = dendritic_yolo.val(data="coco128.yaml", device=device, verbose=False)
    score = float(val_results.box.map50)
    print(f"📊 Validation mAP50: {score:.4f}")
    
    # Add score to tracker - this triggers PAI.png generation
    dendritic_model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
        score, dendritic_model
    )
    
    if restructured:
        print("🔄 Model restructured! Re-initializing optimizer...")
        optimArgs['params'] = dendritic_model.parameters()
        optimizer, scheduler = GPA.pai_tracker.setup_optimizer(
            dendritic_model, optimArgs, schedArgs
        )
        dendritic_model = dendritic_model.to(device)
        dendritic_yolo.model = dendritic_model
    
    if training_complete:
        print("✅ Dendritic training complete!")
    
    print("✅ Validation score added to tracker")
    print("📊 PAI.png should be generated in the PAI/ folder")
else:
    print("⚠️ PerforatedAI not available - skipping add_validation_score")

In [ ]:
# Validate dendritic model
print("📊 Validating dendritic model...")

try:
    dendritic_val = dendritic_yolo.val(
        data="coco128.yaml",
        device=device
    )
    
    # Extract metrics with error handling
    dendritic_metrics = {
        "mAP50": float(dendritic_val.box.map50) if dendritic_val.box.map50 is not None else 0.0,
        "mAP50-95": float(dendritic_val.box.map) if dendritic_val.box.map is not None else 0.0,
        "precision": float(dendritic_val.box.mp) if dendritic_val.box.mp is not None else 0.0,
        "recall": float(dendritic_val.box.mr) if dendritic_val.box.mr is not None else 0.0,
        "params_M": dendritic_total_params / 1e6,
    }
    
except Exception as e:
    print(f"⚠️ Validation failed: {e}")
    # Use baseline metrics as fallback
    dendritic_metrics = baseline_metrics.copy()
    dendritic_metrics["params_M"] = dendritic_total_params / 1e6

# Measure inference speed
try:
    dendritic_metrics["inference_ms"] = measure_inference_speed(dendritic_yolo.model)
except Exception as e:
    print(f"⚠️ Inference speed measurement failed: {e}")
    dendritic_metrics["inference_ms"] = baseline_metrics.get("inference_ms", 0.0)

print(f"\n📊 Dendritic Metrics:")
for key, value in dendritic_metrics.items():
    print(f"   {key}: {value:.4f}")

# Log to W&B if available
try:
    wandb.log({f"dendritic_{k}": v for k, v in dendritic_metrics.items()})
    wandb.finish()
    print("✅ Logged to W&B")
except:
    print("⚠️ W&B logging skipped")

print("\n✅ Dendritic validation complete!")

---
## Section D: Comparison & Results
Compare baseline and dendritic models, generate visualizations.

In [ ]:
# Calculate deltas with error handling
print("📊 Calculating performance deltas...")

deltas = {}
for key in baseline_metrics:
    if key in dendritic_metrics:
        baseline_val = baseline_metrics[key]
        dendritic_val = dendritic_metrics[key]
        
        if baseline_val != 0:
            delta_pct = ((dendritic_val - baseline_val) / baseline_val) * 100
        else:
            delta_pct = 0
        
        deltas[key] = {
            "baseline": baseline_val,
            "dendritic": dendritic_val,
            "delta_pct": delta_pct
        }

# Create comparison DataFrame
if deltas:
    comparison_df = pd.DataFrame(deltas).T
    comparison_df.columns = ["Baseline", "Dendritic", "Delta (%)"]
    
    print("\n" + "="*70)
    print("📊 RESULTS COMPARISON")
    print("="*70)
    print(comparison_df.round(4).to_string())
    print("="*70)
else:
    print("⚠️ No metrics available for comparison")

In [ ]:
# Generate comparison chart with error handling
print("📊 Generating comparison charts...")

try:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Chart 1: mAP Comparison
    metrics_map = ['mAP50', 'mAP50-95']
    metrics_available = [m for m in metrics_map if m in baseline_metrics and m in dendritic_metrics]
    
    if metrics_available:
        x = np.arange(len(metrics_available))
        width = 0.35
        
        baseline_vals = [baseline_metrics[m] for m in metrics_available]
        dendritic_vals = [dendritic_metrics[m] for m in metrics_available]
        
        axes[0].bar(x - width/2, baseline_vals, width, label='Baseline', color='steelblue')
        axes[0].bar(x + width/2, dendritic_vals, width, label='Dendritic', color='coral')
        axes[0].set_ylabel('Score')
        axes[0].set_title('mAP Comparison')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(metrics_available)
        axes[0].legend()
        axes[0].set_ylim(0, max(max(baseline_vals), max(dendritic_vals)) * 1.2)
    else:
        axes[0].text(0.5, 0.5, 'No mAP data\navailable', ha='center', va='center', transform=axes[0].transAxes)
        axes[0].set_title('mAP Comparison')
    
    # Chart 2: Parameters
    if 'params_M' in baseline_metrics and 'params_M' in dendritic_metrics:
        params = [baseline_metrics['params_M'], dendritic_metrics['params_M']]
        colors = ['steelblue', 'coral']
        axes[1].bar(['Baseline', 'Dendritic'], params, color=colors)
        axes[1].set_ylabel('Parameters (Millions)')
        axes[1].set_title('Model Size Comparison')
        for i, v in enumerate(params):
            axes[1].text(i, v + max(params) * 0.02, f'{v:.2f}M', ha='center')
    else:
        axes[1].text(0.5, 0.5, 'No parameter\ndata available', ha='center', va='center', transform=axes[1].transAxes)
        axes[1].set_title('Model Size Comparison')
    
    # Chart 3: Inference Speed
    if 'inference_ms' in baseline_metrics and 'inference_ms' in dendritic_metrics:
        speeds = [baseline_metrics['inference_ms'], dendritic_metrics['inference_ms']]
        axes[2].bar(['Baseline', 'Dendritic'], speeds, color=colors)
        axes[2].set_ylabel('Inference Time (ms)')
        axes[2].set_title('Inference Speed Comparison')
        for i, v in enumerate(speeds):
            axes[2].text(i, v + max(speeds) * 0.02, f'{v:.1f}ms', ha='center')
    else:
        axes[2].text(0.5, 0.5, 'No inference\nspeed data available', ha='center', va='center', transform=axes[2].transAxes)
        axes[2].set_title('Inference Speed Comparison')
    
    plt.tight_layout()
    plt.savefig('comparison_chart.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Comparison chart saved to 'comparison_chart.png'")
    
except Exception as e:
    print(f"⚠️ Chart generation failed: {e}")
    print("Continuing without visualization...")

In [ ]:
# Print final summary with error handling
print("\n" + "="*70)
print("🏆 DENDRITIC YOLOv8 HACKATHON RESULTS SUMMARY")
print("="*70)

try:
    param_reduction = 0
    map50_change = 0
    speed_change = 0
    
    if 'params_M' in baseline_metrics and 'params_M' in dendritic_metrics:
        param_reduction = ((baseline_metrics['params_M'] - dendritic_metrics['params_M']) / baseline_metrics['params_M']) * 100
        
    if 'mAP50' in baseline_metrics and 'mAP50' in dendritic_metrics:
        map50_change = dendritic_metrics['mAP50'] - baseline_metrics['mAP50']
        
    if 'inference_ms' in baseline_metrics and 'inference_ms' in dendritic_metrics:
        speed_change = ((baseline_metrics['inference_ms'] - dendritic_metrics['inference_ms']) / baseline_metrics['inference_ms']) * 100
    
    print(f"\n📦 Parameter Change: {param_reduction:+.1f}%")
    if 'params_M' in baseline_metrics and 'params_M' in dendritic_metrics:
        print(f"   Baseline: {baseline_metrics['params_M']:.2f}M → Dendritic: {dendritic_metrics['params_M']:.2f}M")
    
    print(f"\n🎯 mAP50 Change: {map50_change:+.3f}")
    if 'mAP50' in baseline_metrics and 'mAP50' in dendritic_metrics:
        print(f"   Baseline: {baseline_metrics['mAP50']:.3f} → Dendritic: {dendritic_metrics['mAP50']:.3f}")
    
    print(f"\n⚡ Speed Change: {speed_change:+.1f}%")
    if 'inference_ms' in baseline_metrics and 'inference_ms' in dendritic_metrics:
        print(f"   Baseline: {baseline_metrics['inference_ms']:.1f}ms → Dendritic: {dendritic_metrics['inference_ms']:.1f}ms")
    
    print(f"\n🔧 PerforatedAI Status: {'✅ Available' if PERFORATED_AI_AVAILABLE else '❌ Not Available'}")
    
except Exception as e:
    print(f"⚠️ Error calculating summary: {e}")

print("\n" + "="*70)
print("🔗 Training completed! Check the runs/ directory for training outputs.")
print("="*70)

In [ ]:
# Save results and provide submission summary
print("💾 Saving hackathon results...")

try:
    # Compile comprehensive results
    results = {
        "hackathon": "PerforatedAI Dendritic Optimization Challenge",
        "model": "YOLOv8n",
        "dataset": "COCO128",
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "environment": {
            "device": device,
            "pytorch_version": torch.__version__,
            "cuda_available": torch.cuda.is_available(),
            "perforated_ai_available": PERFORATED_AI_AVAILABLE
        },
        "baseline": baseline_metrics if 'baseline_metrics' in locals() else {},
        "dendritic": dendritic_metrics if 'dendritic_metrics' in locals() else {},
        "improvements": {}
    }
    
    # Calculate improvements if both metrics exist
    if 'baseline_metrics' in locals() and 'dendritic_metrics' in locals():
        improvements = {}
        if 'params_M' in baseline_metrics and 'params_M' in dendritic_metrics:
            improvements["parameter_reduction_pct"] = ((baseline_metrics['params_M'] - dendritic_metrics['params_M']) / baseline_metrics['params_M']) * 100
        if 'mAP50' in baseline_metrics and 'mAP50' in dendritic_metrics:
            improvements["mAP50_change"] = dendritic_metrics['mAP50'] - baseline_metrics['mAP50']
        if 'inference_ms' in baseline_metrics and 'inference_ms' in dendritic_metrics:
            improvements["inference_speedup_pct"] = ((baseline_metrics['inference_ms'] - dendritic_metrics['inference_ms']) / baseline_metrics['inference_ms']) * 100
        
        results["improvements"] = improvements
    
    # Save results to JSON file
    with open('hackathon_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print("✅ Results saved to 'hackathon_results.json'")
    
    # Display summary
    print("\n" + "="*70)
    print("🏆 HACKATHON SUBMISSION SUMMARY")
    print("="*70)
    if results["improvements"]:
        print("📊 Key Improvements with PerforatedAI:")
        for key, value in results["improvements"].items():
            print(f"   • {key}: {value:+.2f}{'%' if 'pct' in key else ''}")
    else:
        print("📋 Results available in JSON format for further analysis")
    
    print(f"\n🔧 Environment: {device.upper()} | PyTorch {torch.__version__}")
    print(f"🧠 PerforatedAI: {'✅ Active' if PERFORATED_AI_AVAILABLE else '❌ Unavailable'}")
    print("="*70)
    
except Exception as e:
    print(f"❌ Error saving results: {e}")
    print("Results data may be incomplete due to training issues")

print("\n🎯 Next steps for submission:")
print("1. Download 'hackathon_results.json' from Files panel")
print("2. Include training outputs from 'runs/' directory")
print("3. Submit with comparison charts and analysis")
print("4. Reference this notebook for methodology")